<a href="https://colab.research.google.com/github/pythonuzgit/Computer-Vision/blob/master/Computer%20Vision/Dog_breed_image_retrieval_dataset_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wolta

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.8 MB/s eta 0:00:00


In [5]:
!unzip '/content/dog-breed-image-retrieval-dataset.zip'

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
  inflating: Dataset-Dog Breed Image Retrieval/train/Dhole/085.jpg  
  inflating: Dataset-Dog Breed Image Retrieval/train/Dhole/086.jpg  
  inflating: Dataset-Dog Breed Image Retrieval/train/Dhole/087.jpg  
  inflating: Dataset-Dog Breed Image Retrieval/train/Dhole/088.jpg  
  inflating: Dataset-Dog Breed Image Retrieval/train/Dhole/089.jpg  
  inflating: Dataset-Dog Breed Image Retrieval/train/Dhole/090.jpg  
  inflating: Dataset-Dog Breed Image Retrieval/train/Dhole/091.jpg  
  inflating: Dataset-Dog Breed Image Retrieval/train/Dhole/092.jpg  
  inflating: Dataset-Dog Breed Image Retrieval/train/Dhole/093.jpg  
  inflating: Dataset-Dog Breed Image Retrieval/train/Dhole/094.jpg  
  inflating: Dataset-Dog Breed Image Retrieval/train/Dhole/095.jpg  
  inflating: Dataset-Dog Breed Image Retrieval/train/Dhole/096.jpg  
  inflating: Dataset-Dog Breed Image Retrieval/train/Dhole/097.jpg  
  inflating: Dataset-Dog Breed Image

In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf

import os
import cv2

from glob import glob

Data Analysis

In [8]:
d_paths = glob('/content/Dataset-Dog Breed Image Retrieval/train/*')

i_paths = []

for d_path in d_paths:
    i_paths.extend(glob('{}/*'.format(d_path)))

print(len(i_paths))

8068


In [7]:
import os
import cv2
from wolta.visual_tools import dataset_size_same
from glob import glob

d_paths = glob('/content/Dataset-Dog Breed Image Retrieval/train/*')

i_paths = []

for d_path in d_paths:
    i_paths.extend(glob('{}/*'.format(d_path)))

print(len(i_paths))

# Filter out invalid image paths
valid_i_paths = []
for path in i_paths:
    if os.path.isfile(path) and cv2.imread(path) is not None: # Check if file exists and can be read by OpenCV
        valid_i_paths.append(path)
    else:
        print(f"Skipping invalid image path: {path}") # Print a warning for skipped paths

# Call dataset_size_same with the filtered paths
dataset_size_same(valid_i_paths)

8068
Skipping invalid image path: /content/Dataset-Dog Breed Image Retrieval/train/Beagle/Afghan


True

Image Processing and Stacking

In [12]:
from wolta.visual_tools import crop

for d_path in d_paths:
    d_name = d_path.split('/')[-1]
    save_dir = '/content/working/{}'.format(d_name)
    os.makedirs(save_dir, exist_ok=True) # Ensure directory creation doesn't raise error if exists

    i_paths = glob('{}/*'.format(d_path))
    for i_path in i_paths:
        i_name = i_path.split('/')[-1]
        img = cv2.imread(i_path)

        # Check if image was loaded successfully
        if img is not None:
            edge = min(img.shape[0], img.shape[1])
            img = crop(img, crop_width=edge, crop_height=edge, get_img=True)
            img = cv2.resize(img, (128, 128))
            cv2.imwrite('{}/{}'.format(save_dir, i_name), img)
        else:
            print(f"Skipping invalid image file: {i_path}") # Print a warning for skipped files

Skipping invalid image file: /content/Dataset-Dog Breed Image Retrieval/train/Beagle/Afghan


Data Splitting

In [13]:
train_ds, test_val_ds = tf.keras.utils.image_dataset_from_directory(
    '/content/working',
    validation_split=0.4,
    subset='both',
    seed=123,
    image_size=(128, 128),
    batch_size=16
)

Found 8067 files belonging to 71 classes.
Using 4841 files for training.
Using 3226 files for validation.


In [14]:
test_val_ds_size = tf.data.experimental.cardinality(test_val_ds).numpy()
test_val_split_size = int(0.5 * test_val_ds_size)

validation_ds = test_val_ds.take(test_val_split_size)
test_ds = test_val_ds.skip(test_val_split_size)

In [15]:
names = train_ds.class_names
num_classes = len(names)

print(names)

['African Wild Dog', 'African Wild Dog - Copy', 'Airedale', 'Airedale - Copy', 'American Hairless', 'American Spaniel', 'Basenji', 'Basset', 'Beagle', 'Bearded Collie', 'Bermaise', 'Bichon Frise', 'Blenheim', 'Bloodhound', 'Bluetick', 'Border Collie', 'Borzoi', 'Boston Terrier', 'Boxer', 'Bull Mastiff', 'Bull Terrier', 'Bulldog', 'Cairn', 'Chihuahua', 'Chinese Crested', 'Chow', 'Clumber', 'Cockapoo', 'Cocker', 'Collie', 'Corgi', 'Coyote', 'Dalmation', 'Dhole', 'Dingo', 'Doberman', 'Elk Hound', 'French Bulldog', 'German Sheperd', 'Golden Retriever', 'Great Dane', 'Great Perenees', 'Greyhound', 'Groenendael', 'Irish Spaniel', 'Irish Wolfhound', 'Japanese Spaniel', 'Komondor', 'Labradoodle', 'Labrador', 'Lhasa', 'Malinois', 'Maltese', 'Mex Hairless', 'Newfoundland', 'Pekinese', 'Pit Bull', 'Pomeranian', 'Poodle', 'Pug', 'Rhodesian', 'Rottweiler', 'Saint Bernard', 'Schnauzer', 'Scotch Terrier', 'Shar_Pei', 'Shiba Inu', 'Shih-Tzu', 'Siberian Husky', 'Vizsla', 'Yorkie']


In [16]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
validation_ds = validation_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

Model

In [17]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [18]:
model = Sequential([
    layers.Input(shape=[128, 128, 3]),
    layers.Rescaling(1./255),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)
])

In [19]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [20]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)                │ (None, 128, 128, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 128, 128, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 64, 64, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 64, 64, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 32, 32, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 32, 32, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 16, 16, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 16384)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       2,097,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 71)                  │           9,159 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,130,023 (8.13 MB)

 Trainable params: 2,130,023 (8.13 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
history = model.fit(
    train_ds,
    validation_data=validation_ds,
    epochs=10
)

Epoch 1/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 92s 304ms/step - accuracy: 0.6968 - loss: 1.0861 - val_accuracy: 0.1702 - val_loss: 4.6951
Epoch 2/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 92s 303ms/step - accuracy: 0.8391 - loss: 0.5615 - val_accuracy: 0.1751 - val_loss: 5.7817
Epoch 3/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 139s 294ms/step - accuracy: 0.9195 - loss: 0.2692 - val_accuracy: 0.1720 - val_loss: 6.8334
Epoch 4/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 144s 300ms/step - accuracy: 0.9537 - loss: 0.1446 - val_accuracy: 0.1714 - val_loss: 7.9074
Epoch 5/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 91s 300ms/step - accuracy: 0.9461 - loss: 0.1789 - val_accuracy: 0.1813 - val_loss: 8.0514
Epoch 6/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 93s 307ms/step - accuracy: 0.9573 - loss: 0.1333 - val_accuracy: 0.1819 - val_loss: 8.6184
Epoch 7/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 140s 300ms/step - accuracy: 0.9592 - loss: 0.1268 - val_accuracy: 0.1689 - val_loss: 8.5289
Epoch 8/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 94s 311ms/step - accuracy: 0.9672 - loss

In [24]:
loss, acc = model.evaluate(test_ds)

print(f"Test Accuracy: {acc * 100:.2f}%")

101/101 ━━━━━━━━━━━━━━━━━━━━ 8s 76ms/step - accuracy: 0.1746 - loss: 9.5697
Test Accuracy: 18.51%
